# Self-Consistent gap equation
$$\Delta = \lambda  \int_{-Ec}^{Ec} \Delta \frac{tanh(\frac {\sqrt{\xi^2 + |\Delta|^2}}{2K_{b}T})}{2\sqrt{\xi^2 + |\Delta|^2}}d\xi $$

In [ ]:
#Including libraries
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
import pandas as pd

#Data generation
def f(E):
  return (lam*delta_in*np.tanh(np.sqrt(E**2 + abs(delta_in)**2)/(2*Kb*T)))/(2*np.sqrt((E**2 + abs(delta_in)**2)))

import timeit
time_list = []

import numpy as np
from pandas.core.sorting import lexsort_indexer
import pandas as pd
#physical parameters of the function
lam_vetor = np.linspace(0.01, 1, 20)
T_vetor = np.linspace(0,90, 100)
Ec_vetor = np.linspace(100,10000,10)
Kb = k
data_frame = pd.DataFrame()
c = 0
array_simple = [[]] #T, lam, Ec, initial, convergence_value
#data generation loop with parameters
for Ec in Ec_vetor:
  for lam in lam_vetor:
    for T in T_vetor:
      i=0
      c+=1
      #bruce force method
      delta_in = 10
      start = timeit.default_timer()#time
      delta_fin = quad(f, -Ec, Ec)
      delta_in_list = [delta_in]
      while (abs(delta_in- delta_fin[0])>1e-4):
        delta_in = delta_fin[0]
        delta_fin = quad(f, -Ec, Ec)
        delta_in_list.append(delta_in)
        i += 1
      #time
      stop = timeit.default_timer()
      time_list.append(stop-start)
      if delta_in_list[-1]>1e-2:#excluding the trivial solution
        array_simple.append([T, lam, Ec, delta_in_list[0], delta_in_list[-1]])
        delta_in_list = np.array(delta_in_list).reshape(-1,1)
        iterations = np.array(range(i+1)).reshape(-1,1)
        final = np.concatenate((iterations,delta_in_list), axis = 1)
        final = pd.DataFrame(final)
        data_frame = pd.concat([data_frame, final], axis=1)

print('Time: ', sum(time_list))  

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

array_simple = pd.read_csv('data (1).csv')
array_simple = pd.DataFrame(array_simple)
array_simple.dropna(inplace = True)
print(array_simple)
Y = array_simple.iloc[:,4]
X = array_simple.iloc[:,0:3]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.4, random_state = 42)#40% for train

#Scaling Data
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.fit_transform(X_test)

#Hyperparameters tuning
n_estimators = [2,3,4,5,6,7,8,9, 20,50,100] # number of trees
max_features = ['auto', 'sqrt'] # max number of features for each split
max_depth = [int(x) for x in np.linspace(4, 30, num = 1)] #length for each tree
min_samples_split = [2, 4, 6,8, 10] # min number of features for each split
min_samples_leaf = [1, 2,3, 4] # min number of samples leaf
bootstrap = [True, False] 

#grid to search the best parameters
random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap,}


rf = RandomForestRegressor()

rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

rf_random.fit(X_train, Y_train)

print ('Random grid: ', random_grid, '\n')
# print the best parameters
print ('Best Parameters: ', rf_random.best_params_, ' \n')

#using the best model
randmf = RandomForestRegressor(n_estimators =50, min_samples_leaf=4, min_samples_split = 2,  max_features = 'auto', max_depth= 5, bootstrap=True)  
randmf.fit( X_train, Y_train) 

import timeit
from sklearn.metrics import mean_absolute_percentage_error

start = timeit.default_timer()

pred_i=randmf.predict( X_test) 
stop = timeit.default_timer()

print('Time: ', stop - start)  
print('time per test: ', (stop - start)/X_test.shape[0])

mean_absolute_percentage_error(Y_test, pred_i)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(Y_test, pred_i)
rmse = np.sqrt(mse)
print(rmse) 
from sklearn.metrics import r2_score
r2_score(Y_test, pred_i)
y_train_predict = randmf.predict(X_train) 
print(r2_score(Y_test, pred_i), r2_score(Y_train, y_train_predict))

plt.scatter(Y_test, pred_i, marker=3)
plt.plot(Y_test, Y_test, color ='r')
plt.ylabel('predicted')
plt.xlabel('Y_real')
plt.show()

from sklearn.model_selection import cross_val_score,KFold

kf=KFold(n_splits=5)
score_train=cross_val_score(randmf,X_train,Y_train,cv=kf)
#cross validation
from sklearn.model_selection import cross_val_score,KFold
kf=KFold(n_splits=5)
score_test=cross_val_score(randmf,X_test,Y_test,cv=kf)

plt.plot(score_train, label = 'treino')
plt.plot(score_test, label = 'teste')
plt.title('Cross Validation')
plt.legend()
plt.show()